<a href="https://colab.research.google.com/github/esusrutha/zeroshot/blob/main/KnowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=6dacda606177da04499b6ef63b5b5a5ef285fd6c42ca1cf27cf7842ce3334001
  Stored in directory: /root/.cache/pip/wheels/cd/7d/6b/17150450b80b4a3656a84330e22709ccd8dc0f8f4773ba4133
Successfully built torch_geometric


In [3]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=91066025048cd3c7fda06b6e72dc150c24eca1884e2bef06a4d7df9f6034bf13
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [5]:
#Fixed code for S-BERT-KG embedding alignment
from sentence_transformers import SentenceTransformer
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd

#Load the data from two csv files and merge them into a single dataframe
RESULTS_DIR = 'drive/MyDrive'
tweets_df1 = pd.read_csv(RESULTS_DIR + '/D1.csv', encoding='latin1', engine='python')
tweets_df2 = pd.read_csv(RESULTS_DIR + '/D22.csv', encoding='latin1', engine='python')
df = pd.concat([tweets_df1, tweets_df2], ignore_index=True)

#Load pre-trained S-BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

#Define a function to embed a text using S-BERT
def embed_text(text):
  # Embed the text using S-BERT
  if isinstance(text, str):
      embeddings = model.encode(text)
  else:
    embeddings = model.encode([t for t in text if isinstance(t, str)])
    # Convert the embeddings to a PyTorch tensor
    tensor = torch.FloatTensor(embeddings)
  # Return the tensor
  return tensor

#Define a function to create a PyTorch Geometric graph from a knowledge graph
def create_kg_graph(kg):
  # Get the nodes and edges from the knowledge graph
  nodes = list(kg.keys())
  edges = []
  for node1 in kg.keys():
    for node2 in kg[node1]:
      if isinstance(node2, list):
        node2 = ', '.join(node2)
      if node1 in nodes and node2 in nodes:
        edges.append((nodes.index(node1), nodes.index(node2)))
        # Create the PyTorch Geometric graph data object
        data = Data(x=torch.eye(len(nodes)), edge_index=torch.LongTensor(edges).t())
  # Return the graph data object
  return data

#Define a knowledge graph
kg = {
'tweet': ['Advice','China','Mask','News','Transportation','USA','Vaccine'],
'Advice': ['Stay at home','wash hands','wear mask','social distancing'],
'China': ['Wuhan','China Coronavirus Updates','China news','other tweets related to China'],
'Mask': ['Mask shortage','wear mask','mask types','N50','N95','3M8210','3M9001','3M9322','3M9501'],
'News': ['Coronavirus updates','news','rules'],
'Transportation': ['Flights','traffic','traveling'],
'USA': ['U.S. Coronavirus Updates','COVID19','U.S. news','United States','US','USA'],
'Vaccine': ['Vaccine news','vaccine progress','vaccine injection'],
}
def create_kg_graph(kg, embeddings):
  # Get the nodes and edges from the knowledge graph
  nodes = list(kg.keys())
  edges = []
  for node1 in kg.keys():
    for node2 in kg[node1]:
      if isinstance(node2, list):
        node2 = ', '.join(node2)
      if node1 in nodes and node2 in nodes:
        edges.append((nodes.index(node1), nodes.index(node2)))
  # Create the PyTorch Geometric graph data object
  x = torch.FloatTensor(embeddings)
  data = Data(x=x, edge_index=torch.LongTensor(edges).t())
  return data
#Embed the text and create the PyTorch Geometric graph data object
text = df["hashtags"].apply(lambda x: ', '.join(x.strip("[]").split(", ")) if isinstance(x, str) else '')
embeddings = embed_text(text)
graph_data = create_kg_graph(kg, embeddings)


#Define a GCN model
class GCN(torch.nn.Module):
  def __init__(self, in_channels, out_channels):
    super(GCN, self).__init__()
    self.conv1 = GCNConv(in_channels=in_channels, out_channels=out_channels)
    self.conv2 = GCNConv(in_channels=out_channels, out_channels=out_channels)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.conv2(x, edge_index)
    return x
   
#Define a function  to train the GCN model
def train_gcn(model, data, embeddings, epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  criterion = torch.nn.MSELoss()
  for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = criterion(output, embeddings)
    loss.backward()
    optimizer.step()
  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

#Embed the text and create the PyTorch Geometric graph data object
text = df["hashtags"].apply(lambda x: ', '.join(x.strip("[]").split(", ")) if isinstance(x, str) else '')

embeddings = embed_text(text)

#Reshape the embeddings tensor to have shape (num_tweets, embedding_size)

embeddings = embeddings.reshape(len(text), embeddings.shape[1])

graph_data = create_kg_graph(kg, embeddings)

#Train the GCN model
model = GCN(embeddings.shape[1], embeddings.shape[1])
train_gcn(model, graph_data, embeddings, 100)

#Get the aligned embeddings
aligned_embeddings = model(graph_data.x, graph_data.edge_index)
print(aligned_embeddings)

Epoch 100/100, Loss: 0.18791426718235016
tensor([[ 0.1262, -0.2471,  0.7686,  ..., -0.0542,  0.0829, -0.0556],
        [ 0.1262, -0.2471,  0.7686,  ..., -0.0542,  0.0829, -0.0556],
        [ 0.1262, -0.2471,  0.7686,  ..., -0.0542,  0.0829, -0.0556],
        ...,
        [ 0.1262, -0.2471,  0.7686,  ..., -0.0542,  0.0829, -0.0556],
        [ 0.1262, -0.2471,  0.7686,  ..., -0.0542,  0.0829, -0.0556],
        [ 0.1262, -0.2471,  0.7686,  ..., -0.0542,  0.0829, -0.0556]],
       grad_fn=<AddBackward0>)
